In [5]:
import pandas as pd
X=pd.read_csv("crop_yield.csv")
X.iloc[:20]

,Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest,Yield_tons_per_hectare
0,West,Sandy,Cotton,897.077239,27.676966,False,True,Cloudy,122,6.555816
1,South,Clay,Rice,992.673282,18.026142,True,True,Rainy,140,8.527341
2,North,Loam,Barley,147.998025,29.794042,False,False,Sunny,106,1.127443
3,North,Sandy,Soybean,986.866331,16.644190,False,True,Rainy,146,6.517573
4,South,Silt,Wheat,730.379174,31.620687,True,True,Cloudy,110,7.248251
5,South,Silt,Soybean,797.471182,37.704974,False,True,Rainy,74,5.898416
6,West,Clay,Wheat,357.902357,31.593431,False,False,Rainy,90,2.652392
7,South,Sandy,Rice,441.131154,30.887107,True,True,Sunny,61,5.829542
8,North,Silt,Wheat,181.587861,26.752729,True,False,Sunny,127,2.943716
9,West,Sandy,Wheat,395.048968,17.646199,False,True,Rainy,140,3.707293


In [24]:
X_mod=X.drop("Days_to_Harvest",axis=1) #personally feel its a redundant column with no particular effect on the prediction
pred=X_mod.Yield_tons_per_hectare
X_t=X_mod.drop("Yield_tons_per_hectare",axis=1)
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid=train_test_split(X_t,pred,test_size=0.2)

In [29]:
#main preprocessing
#columns - Region,Soil_type,Crop,Fertilizer_Used,Irrigation_Used,Weather condition contain categorical variables
#columns Fertilizer and Irrigation will be implemented using ordinal encoding so as to maintain simplicity (since they are essentially true/false values)
#rest will be implemented using one hot encoding
List1=[]
List2=[]
for col in X_mod.columns:
    if X_mod[col].dtype=="object" or X_mod[col].dtype=="bool":
      if col=="Fertilizer_Used" or col=="Irrigation_Used":
        List2.append(col)
      else:
        List1.append(col)
#list 1 now contains all columns in the dataframe which have entries of object datatype in them i.e these are the columns containing categorical variables that are to be one hot encoded
X1=X_train.drop(List1,axis=1)
X2=X_valid.drop(List1,axis=1)
from sklearn.preprocessing import OneHotEncoder
OHE=OneHotEncoder(sparse_output=False)
P1=pd.DataFrame(OHE.fit_transform(X_train[List1]))
P2=pd.DataFrame(OHE.transform(X_valid[List1])) #using fit here causes data leakage as the model can now peek into validation data. so fit is not being used here.
P1.index=X_train.index
P2.index=X_valid.index
X_train_final=pd.concat([X1,P1],axis=1)
X_valid_final=pd.concat([X2,P2],axis=1)
from sklearn.preprocessing import OrdinalEncoder
#list 2 now contains all columns in the dataframe which have entries of object datatype in them i.e these are the columns containing categorical variables that are to be ordinal encoded
OE=OrdinalEncoder()
X_train_final[List2]=OE.fit_transform(X_train_final[List2])
X_valid_final[List2]=OE.transform(X_valid_final[List2]) #fit is not being used here for the same reason mentioned above
X_train_final.head()

,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,0,1,2,3,4,5,...,9,10,11,12,13,14,15,16,17,18
503435,256.101679,35.232440,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
129059,551.047154,25.332282,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
189298,795.575617,29.696078,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
803856,757.944090,32.071789,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
101806,478.369058,27.206541,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [31]:
#modelling using gradient boosting
from xgboost import XGBRegressor
model=XGBRegressor(n_estimators=1000,learning_rate=0.05,n_jobs=-1,random_state=0)
model.fit(X_train_final,y_train)
print(model.predict(X_valid_final))
model.score(X_valid_final,y_valid)

[4.488103  3.7954645 4.787093  ... 5.573222  3.6549723 2.8977602]


0.912763062266404

In [33]:
model.score(X_train_final, y_train)
#the training and validation scores being really close to each other implies the absence of significant overfitting
#Care has been taken to ensure that there is no validation data being leaked as mentioned in the previous steps

0.9149505970190712